In [1]:
# Prueba - Python para el análisis de datos

In [2]:
# Antes de comenzar la prueba, importaremos las librerias necesarias y crearemos una conexion a la base de datos de PostgreSQL.

import numpy as np                                      # Importamos numpy y pandas.
import pandas as pd 

import psycopg2                                         # Importamos psycopg2 y sqlalchemy para conectarnos a postgresql.                 
from sqlalchemy import create_engine, text

# Definimos credenciales de la base de datos.

usuario_db = "postgres"          
password_db = "juano321" 
host_db = "localhost"            
puerto_db = "5432"               
base = "classicmodels"          

# Creamos string de conexión en formato que entiende SQLAlchemy.

db_url = "postgresql+psycopg2://"+usuario_db+":"+password_db+"@"+host_db+":"+puerto_db+"/"+base

# Creamos engine para conectarnos a PostgreSQL.

engine = create_engine(db_url)

# Traemos todos los registros de la tabla 'customers' a un DataFrame.

query = "SELECT * FROM customers"
df = pd.read_sql(query, engine)

# Mostramos los primeros registros.

df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0


El área comercial de una empresa pide realizar un cierre de año de las ventas, tanto para
revisar si las metas fueron cumplidas, como para poder planificar el siguiente año. Para ello,
considerarán los datos del dataset classicmodels.sql para responder algunas preguntas,
realizando las siguientes tareas.

In [3]:
""" 
1. Genera una función llamada leer_tabla(tabla, engine) y utilízala para leer tablas
completas desde la base de datos en dataframes independientes. Utilizando esta
función, importa las siguientes tablas:

- order
- orderdetails
- customers
- products
- employees
"""


# Función para leer tablas completas desde la base de datos

def leer_tabla(tabla, engine):
    """
    Lee una tabla completa desde la base de datos PostgreSQL 
    y la devuelve como un DataFrame de pandas.
    
    Parámetros:
        tabla (str): Nombre de la tabla en la base de datos.
        engine: Conexión a la base de datos creada con SQLAlchemy.
        
    Retorna:
        DataFrame con el contenido de la tabla.
    """
    query = f"SELECT * FROM {tabla};"
    return pd.read_sql(query, engine)

# Uso de la función para importar las tablas solicitadas.

orders = leer_tabla("orders", engine)
orderdetails = leer_tabla("orderdetails", engine)
customers = leer_tabla("customers", engine)
products = leer_tabla("products", engine)
employees = leer_tabla("employees", engine)

# Verifico que las tablas se importaron correctamente.

print(orders.head())
print(orderdetails.head())
print(customers.head())
print(products.head())
print(employees.head())

   orderNumber   orderDate requiredDate shippedDate   status  \
0        10100  2003-01-06   2003-01-13  2003-01-10  Shipped   
1        10101  2003-01-09   2003-01-18  2003-01-11  Shipped   
2        10102  2003-01-10   2003-01-18  2003-01-14  Shipped   
3        10103  2003-01-29   2003-02-07  2003-02-02  Shipped   
4        10104  2003-01-31   2003-02-09  2003-02-01  Shipped   

                 comments  customerNumber  
0                    None             363  
1  Check on availability.             128  
2                    None             181  
3                    None             121  
4                    None             141  
   orderNumber productCode  quantityOrdered  priceEach  orderLineNumber
0        10100    S18_1749               30     136.00                3
1        10100    S18_2248               50      55.09                2
2        10100    S18_4409               22      75.46                4
3        10100    S24_3969               49      35.29         

In [4]:
""" 
2. Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el
parámetro validate para asegurar la integridad referencial.
"""


# Primero verificamos las columnas de cada DataFrame para identificar las claves foráneas y primarias.

print("Columnas de cada DataFrame:")
print("Orders:", orders.columns)
print("Order Details:", orderdetails.columns)
print("Customers:", customers.columns)
print("Products:", products.columns)
print("Employees:", employees.columns)  

Columnas de cada DataFrame:
Orders: Index(['orderNumber', 'orderDate', 'requiredDate', 'shippedDate', 'status',
       'comments', 'customerNumber'],
      dtype='object')
Order Details: Index(['orderNumber', 'productCode', 'quantityOrdered', 'priceEach',
       'orderLineNumber'],
      dtype='object')
Customers: Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit'],
      dtype='object')
Products: Index(['productCode', 'productName', 'productLine', 'productScale',
       'productVendor', 'productDescription', 'quantityInStock', 'buyPrice',
       'MSRP'],
      dtype='object')
Employees: Index(['employeeNumber', 'lastName', 'firstName', 'extension', 'email',
       'officeCode', 'reportsTo', 'jobTitle'],
      dtype='object')


In [5]:
# Ahora realizamos los cruces entre los DataFrames utilizando el parametro validate para asegurar la integridad referencial.

# Orders con Customers (muchos pedidos pertenecen a un cliente → m:1).

orders_customers = orders.merge(
    customers,
    on="customerNumber",
    how="inner",
    validate="m:1"
)

# Orders con OrderDetails (un pedido tiene varios detalles → 1:m).

orders_details = orders.merge(
    orderdetails,
    on="orderNumber",
    how="inner",
    validate="1:m"
)

# OrderDetails con Products (muchos detalles corresponden a un producto → m:1).

details_products = orderdetails.merge(
    products,
    on="productCode",
    how="inner",
    validate="m:1"
)

# Customers con Employees (muchos clientes son atendidos por un representante → m:1).

customers_employees = customers.merge(
    employees,
    left_on="salesRepEmployeeNumber",
    right_on="employeeNumber",
    how="inner",
    validate="m:1"
)

""" *Observacion sobre el cruce con la tabla employees*

En el merge entre customers y employees, utilice un inner join para asegurar la integridad referencial, es decir, que solo se consideren aquellos clientes que efectivamente tienen un representante de ventas asignado. 
Esto cumple con lo solicitado en la prueba.

Sin embargo, es importante destacar que en la base de datos existen clientes que no tienen registrado un salesRepEmployeeNumber. Al usar inner, estos clientes quedan fuera del df_final.
En un escenario real, podria ser recomendable utilizar un left join, de manera que se mantengan todos los clientes (incluyendo aquellos sin representante), y manejar los valores faltantes en las columnas correspondientes a empleados."""


"""Finalmente, unimos todos los DataFrames en uno solo, asegurando que todas las relaciones se mantengan.
Esto nos dará un DataFrame final que contiene toda la información relacionada."""

df_final = (orders
    .merge(customers, on="customerNumber", validate="m:1")
    .merge(orderdetails, on="orderNumber", validate="1:m")
    .merge(products, on="productCode", validate="m:1")
    .merge(employees, left_on="salesRepEmployeeNumber", right_on="employeeNumber", validate="m:1")
)

# Verificamos el DataFrame final.

df_final.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,...,buyPrice,MSRP,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,86.70,170.00,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,33.30,60.54,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,43.26,92.03,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,21.75,41.03,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep
4,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,...,58.48,127.13,1504,Jones,Barry,x102,bjones@classicmodelcars.com,7,1102.0,Sales Rep


In [6]:
"""
3. Agrega las siguientes columnas, considerando su nombre y la fórmula asociada.
- venta: quantityOrdered*priceEach.
- costo: quantityOrdered*buyPrice.
- ganancia: considerando las columnas anteriores.
"""

# Calculamos las columnas adicionales requeridas en el DataFrame final.

df_final['venta'] = df_final['quantityOrdered'] * df_final['priceEach']
df_final['costo'] = df_final['quantityOrdered'] * df_final['buyPrice']
df_final['ganancia'] = df_final['venta'] - df_final['costo']        

# Verificamos el DataFrame final con las nuevas columnas.

df_final.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,...,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,venta,costo,ganancia
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep,4080.00,2601.00,1479.00
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep,2754.50,1665.00,1089.50
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep,1660.12,951.72,708.40
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,Online Diecast Creations Co.,Young,Dorothy,...,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep,1729.21,1065.75,663.46
4,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,...,Jones,Barry,x102,bjones@classicmodelcars.com,7,1102.0,Sales Rep,2701.50,1462.00,1239.50


In [7]:
# 4. ¿Cuál fue el total de ventas por línea de productos? Incluye una fila de totales.

# Agrupar por producto.

ventas_por_linea_producto = df_final.groupby('productLine')["venta"].sum().reset_index()

# Renombrar columna a 'total_ventas'.

ventas_por_linea_producto = ventas_por_linea_producto.rename(columns={'venta': 'total_ventas'})

# Agregar fila con total general.

total_general = pd.DataFrame({
    "productLine": ["Total"],
    "total_ventas": [ventas_por_linea_producto['total_ventas'].sum()]
})
ventas_por_linea_producto = pd.concat([ventas_por_linea_producto, total_general], ignore_index=True)

# Mostrar las primeras filas.

ventas_por_linea_producto.head()

,productLine,total_ventas
0,Classic Cars,3853922.49
1,Motorcycles,1121426.12
2,Planes,954637.54
3,Ships,663998.34
4,Trains,188532.92


In [8]:
# 5. ¿Cuántos clientes distintos hicieron compras?.

clientes_distintos = df_final['customerNumber'].nunique()
print(f"Cantidad de clientes distintos que hicieron compras: {clientes_distintos}") 

Cantidad de clientes distintos que hicieron compras: 98


In [9]:
# 6. ¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?.

# Clientes que sí hicieron pedidos.

clientes_con_pedidos = orders["customerNumber"].unique()

# Filtramos los que no estan en la lista anterior.

clientes_sin_pedidos = customers[~customers["customerNumber"].isin(clientes_con_pedidos)]

# Contamos cuantos son y mostramos algunos ejemplos.

if len(clientes_sin_pedidos) > 0:
    print(f"Sí hay clientes que aún no han comprado. Son {len(clientes_sin_pedidos)} en total.")
else:
    print("Todos los clientes han realizado al menos una compra.")

clientes_sin_pedidos.head()

Sí hay clientes que aún no han comprado. Son 24 en total.


,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
6,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,None,Warszawa,None,01-012,Poland,NaN,0.0
20,168,American Souvenirs Inc,Franco,Keith,2035557845,149 Spinnaker Dr.,Suite 101,New Haven,CT,97823,USA,1286.0,0.0
21,169,Porto Imports Co.,de Castro,Isabel,(1) 356-5555,Estrada da saúde n. 58,None,Lisboa,None,1756,Portugal,NaN,0.0
36,206,"Asian Shopping Network, Co",Walker,Brydey,+612 9411 1555,Suntec Tower Three,8 Temasek,Singapore,None,038988,Singapore,NaN,0.0
41,223,Natürlich Autos,Kloss,Horst,0372-555188,Taucherstraße 10,None,Cunewalde,None,01307,Germany,NaN,0.0


In [10]:
"""
Para este punto debes aplicar el principio DRY, por lo que se deben utilizar funciones para
realizar el filtrado por fechas, generar tablas pivote y escribir tabla en Postgre. Las funciones
deben estar en un archivo separado llamado funciones.py y ser importadas al Jupyter
Notebook. En este archivo se debe incluir:

● Una función que permita filtrar un DataFrame por fechas, indicando dataframe,
columna para filtrar, fecha inicio y fecha fin. La función debe retornar un DataFrame.

● Una función que permita generar reportes dependiendo de parámetros de entrada
como dataframe, filas, columnas, valores y medida (funcion_agrupadora). Utilizar
fill_value = 0. Esta función debe retornar un DataFrame pivotado.

● Una función que permita escribir en la base de datos a través del guardado de un
DataFrame dependiendo de parámetros de entrada como DataFrame, nombre de la
tabla, engine y comportamiento en caso de que exista la tabla (if_exists).
""" 

# Importamos las funciones desde el archivo funciones.py.

from funciones import filtrar_por_fechas, generar_reporte, escribir_en_base_de_datos

In [11]:
"""
7. Se solicita la creación de dos reportes, que respondan las preguntas dadas

● ¿Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante
el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
top_10_clientes_2005, en la que se especifique el nombre del cliente y su
correspondiente venta, costo y ganancia.
""" 

# Filtramos el año 2005 con la funcion importada.

df_2005 = filtrar_por_fechas(
    df=df_final,                    # DataFrame completo.  
    columna_fecha='orderDate',      # Columna que contiene las fechas.
    fecha_inicio='2005-01-01',      # Fecha inicial.
    fecha_fin='2005-12-31'          # Fecha final.
    ).copy()                        # Evitamos SettingWithCopyWarning creando una copia con .copy().

# Creamos columnas de cálculo

df_2005['venta'] = df_2005['quantityOrdered'] * df_2005['priceEach']
df_2005['costo'] = df_2005['quantityOrdered'] * df_2005['buyPrice']
df_2005['ganancia'] = df_2005['venta'] - df_2005['costo']

# Generamos el reporte de los 10 clientes con mayores ventas brutas en dinero durante 2005.

reporte_clientes = generar_reporte(
    df=df_2005,                                     # DataFrame completo. 
    filas=['customerName'],                         # agrupamos por cliente.
    columnas=None,                                  # sin columnas adicionales.
    valores=['venta', 'costo', 'ganancia'],         # métricas a calcular.
    funcion_agrupadora='sum'                        # sumamos los valores.
).reset_index()

# Ordenar y seleccionar top 10.

top_10_clientes_2005 = reporte_clientes.sort_values(
    by='venta', 
    ascending=False
    ).head(10)

# Guardar en PostgreSQL.

escribir_en_base_de_datos(top_10_clientes_2005, 'top_10_clientes_2005', engine, if_exists='replace')

# Leer la tabla top_10_clientes_2005 desde PostgreSQL.

df_clientes = pd.read_sql("SELECT * FROM top_10_clientes_2005;", engine)
display(df_clientes)

DataFrame escrito en la tabla 'top_10_clientes_2005' con if_exists='replace'


,customerName,costo,ganancia,venta
0,Euro+ Shopping Channel,169989.97,120028.55,290018.52
1,Mini Gifts Distributors Ltd.,115084.72,77397.01,192481.73
2,La Rochelle Gifts,55527.04,35620.07,91147.11
3,The Sharp Gifts Warehouse,50843.02,33141.87,83984.89
4,"Down Under Souveniers, Inc",46389.52,28630.61,75020.13
5,"Anna's Decorations, Ltd",35414.90,21517.40,56932.30
6,Salzburg Collectables,33536.26,18883.81,52420.07
7,Gifts4AllAges.com,33221.25,17585.60,50806.85
8,Corporate Gift Ideas Co.,28561.31,18220.35,46781.66
9,"Oulu Toy Supplies, Inc.",27493.61,19276.91,46770.52


In [12]:
""" 
● ¿Cuál fue el top 10 de artículos más vendidos durante el año 2005? Genera un
DataFrame y guárdalo en una tabla de Postgre llamada top_10_productos_2005, en la
que se especifique el nombre del producto y su correspondiente venta, costo y
ganancia.
"""

# Filtramos el año 2005 con la funcion importada.

df_2005 = filtrar_por_fechas(
    df=df_final,                    # DataFrame completo.
    columna_fecha='orderDate',      # Columna que contiene las fechas.
    fecha_inicio='2005-01-01',      # Fecha inicial.
    fecha_fin='2005-12-31'          # Fecha final.
    ).copy()                        # Evitamos SettingWithCopyWarning creando una copia con .copy().


# Creamos columnas de cálculo.

df_2005['venta'] = df_2005['quantityOrdered'] * df_2005['priceEach']
df_2005['costo'] = df_2005['quantityOrdered'] * df_2005['buyPrice']
df_2005['ganancia'] = df_2005['venta'] - df_2005['costo']

# Usamos generar_reporte agrupando por producto.

reporte_productos = generar_reporte(
    df=df_2005,
    filas=['productName'],                  # agrupamos por producto.
    columnas=None,
    valores=['venta', 'costo', 'ganancia'],
    funcion_agrupadora='sum'
).reset_index()

# Ordenamos por venta y seleccionamos top 10.

top_10_productos_2005 = reporte_productos.sort_values(
    by='venta', 
    ascending=False
    ).head(10)

# Guardamos en PostgreSQL.

escribir_en_base_de_datos(top_10_productos_2005, 'top_10_productos_2005', engine, if_exists='replace')

# Leer la tabla top_10_productos_2005 desde PostgreSQL.

df_productos = pd.read_sql("SELECT * FROM top_10_productos_2005;", engine)
display(df_productos)

DataFrame escrito en la tabla 'top_10_productos_2005' con if_exists='replace'


,productName,costo,ganancia,venta
0,1992 Ferrari 360 Spider red,27031.30,25946.98,52978.28
1,2003 Harley-Davidson Eagle Drag Bike,19023.18,15832.94,34856.12
2,1952 Alpine Renault 1300,17152.92,16884.28,34037.20
3,1968 Ford Mustang,17161.20,14538.31,31699.51
4,1956 Porsche 356A Coupe,25066.50,6365.64,31432.14
5,2002 Suzuki XREO,15308.37,15125.72,30434.09
6,1969 Dodge Charger,15974.56,13592.71,29567.27
7,1997 BMW R 1100 S,16493.06,12254.63,28747.69
8,1917 Grand Touring Sedan,16386.30,11444.40,27830.70
9,1972 Alfa Romeo GTA,18678.24,8547.60,27225.84
